In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install category_encoders
# !pip install dacon_submit_api-0.0.4-py3-none-any.whl
# !pip install mljar-supervised

In [3]:
## import 
import pandas as pd
import numpy as np
from supervised.automl import AutoML

# vis
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
import matplotlib
%matplotlib inline
# matplotlib.rcParams['font.family'] = 'Malgun Gothic' # 한글 패치. 코랩에선 주석하셈
import seaborn as sns

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
# Modeling
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier, XGBRegressor, XGBRFRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, VotingRegressor 
from sklearn.ensemble import StackingClassifier, StackingRegressor
from sklearn.base import ClassifierMixin
# Evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss,mean_squared_error

# CatBoost
from catboost import CatBoostRegressor

# Utility
import os
import time
import datetime # ⚠️2019년 12월30일과 31일의 week of year가 1인 오류가 있음
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
# import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean
import holidays

# 시드값 고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(2023)

# 데이터프레임 컬럼 전체 보기
pd.set_option('display.max_columns', None)

In [4]:
# 데이터 로드

path = '../data/daegu/automl/'

train_org = pd.read_csv(path+'train.csv')
test_org = pd.read_csv(path+'test.csv')

In [5]:
# 이희원님 코드 그대로 사용
for df in [train_org, test_org]:
    # 날짜 데이터
    time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})'

    df[['연', '월', '일', '시간']] = df['사고일시'].str.extract(time_pattern)
    df[['연', '월', '일', '시간']] = df[['연', '월', '일', '시간']].apply(pd.to_numeric) # 추출된 문자열을 수치화해줍니다
    df['사고일시'] = pd.to_datetime(df['사고일시'])
    # df= df.drop(columns=['사고일시']) # 일단 냄겨두자

    # 도시/구/동 분리
    location_pattern = r'(\S+) (\S+) (\S+)'

    df[['도시', '구', '동']] = df['시군구'].str.extract(location_pattern)
    # df= df.drop(columns=['시군구']) # 얘도 일단 냄겨놔

    # 도로형태도 분류 코드
    #road_pattern = r'(.+) - (.+)'

    #df[['도로형태1', '도로형태2']] = df['도로형태'].str.extract(road_pattern)
    #df= df.drop(columns=['도로형태']) # 얘는 드롭함

    # 시간 >> 코사인 변환
    df['cos_hour'] = np.cos(2 * np.pi * df['시간'] / 24)

    # 공휴일 (주말과 공휴일을 묶음)
    # 공휴일 = 1, 비공휴일 = 0
    holi_weekday = ['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01', '2019-05-05', '2019-05-12', '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25',
                '2020-01-01' ,'2020-01-24' ,'2020-01-25', '2020-01-26', '2020-03-01', '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25',
                '2021-01-01' ,'2021-02-11' ,'2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25',
                '2022-01-01' ,'2022-01-31' ,'2022-02-01', '2022-02-02', '2022-03-01', '2022-05-05', '2022-05-08', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2020-10-10', '2022-12-25',
                '2023-01-01' ,'2023-01-21' ,'2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01']
    df['dow'] = df['사고일시'].dt.dayofweek #임시로 만든 컬럼
    df['holiday'] = np.where((df['dow']>=5) | (df['사고일시'].dt.strftime('%Y-%m-%d').isin(holi_weekday)),1,0)
    df.drop('dow',axis=1,inplace=True)
    
    # 계절
    def group_season(df):
        df.loc[(df['월'] == 3) | (df['월'] == 4) | (df['월'] == 5), 'season'] = '봄'
        df.loc[(df['월'] == 6) | (df['월'] == 7) | (df['월'] == 8), 'season'] = '여름'
        df.loc[(df['월'] == 9) | (df['월'] == 10) | (df['월'] == 11), 'season'] = '가을'
        df.loc[(df['월'] == 12) | (df['월'] == 1) | (df['월'] == 2), 'season'] = '겨울'
        return df['season']
    df['season'] = group_season(df)

In [6]:
train_df = train_org
test_df = test_org

In [7]:
# load data
light_df = pd.read_csv(path+'동별_보안등개수.csv')
parking_df = pd.read_csv(path+'주차장급지구분별개수.csv')
speed_df = pd.read_csv(path+'동별_제한속도.csv')
cross_walk_df = pd.read_csv(path+'동별_횡단보도개수.csv')
child_area_df = pd.read_csv(path+'동별_어린이보호구역개수.csv')


# merge data
train_df = pd.merge(train_df, light_df, how='left', on=['구','동'])
train_df = pd.merge(train_df, parking_df, how='left', on=['구','동'])
train_df = pd.merge(train_df, speed_df, how='left', on='동')
train_df = pd.merge(train_df, cross_walk_df, how='left', on='동')
train_df = pd.merge(train_df, child_area_df, how='left', on=['구','동'])

test_df = pd.merge(test_df, light_df, how='left', on=['구','동'])
test_df = pd.merge(test_df, parking_df, how='left', on=['구','동'])
test_df = pd.merge(test_df, speed_df, how='left', on='동')
test_df = pd.merge(test_df, cross_walk_df, how='left', on='동')
test_df = pd.merge(test_df, child_area_df, how='left', on=['구','동'])


In [8]:
if '어린이보호구역개수_x' in train_df.columns:
    # 왜 그런지는 모르겠는데 어린이보호구역개수가 이상하게 들어감
    train_df=train_df.rename(columns={'어린이보호구역개수_x':'어린이보호구역개수'}).drop('어린이보호구역개수_y',axis=1)
    test_df=test_df.rename(columns={'어린이보호구역개수_x':'어린이보호구역개수'}).drop('어린이보호구역개수_y',axis=1)


In [9]:
### 피처 추가...

# 시군구별 사고횟수 >> 기본 코드 사용
accident_counts = train_df['시군구'].value_counts().reset_index()
accident_counts.columns = ['시군구', '사고횟수']
# '시군구'별 사고 횟수를 train_df에 매핑하기 위해 '시군구' 컬럼을 기준으로 사고 횟수를 가져옵니다.
train_df['사고발생횟수'] = train_df['시군구'].map(accident_counts.set_index('시군구')['사고횟수'])
test_df['사고발생횟수'] = test_df['시군구'].map(accident_counts.set_index('시군구')['사고횟수'])




In [10]:
# 점검
display(train_df.head(2))
display(test_df.head(2))

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO,연,월,일,시간,도시,구,동,cos_hour,holiday,season,설치개수,급지구분_1,급지구분_2,급지구분_3,제한속도,횡단보도개수,어린이보호구역개수,사고발생횟수
0,ACCIDENT_00000,2019-01-01,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,상해없음,보행자,여,70세,중상,0,1,0,0,5,2019,1,1,0,대구광역시,중구,대신동,1.0,1,겨울,391.0,11.0,0.0,0.0,40.0,5929.0,2.0,131
1,ACCIDENT_00001,2019-01-01,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,승용,남,39세,상해없음,보행자,남,61세,경상,0,0,1,0,3,2019,1,1,0,대구광역시,달서구,감삼동,1.0,1,겨울,941.0,0.0,1.0,3.0,52.5,10000.0,NaN,678


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,연,월,일,시간,도시,구,동,cos_hour,holiday,season,설치개수,급지구분_1,급지구분_2,급지구분_3,제한속도,횡단보도개수,어린이보호구역개수,사고발생횟수
0,ACCIDENT_39609,2022-01-01 01:00:00,토요일,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람,2022,1,1,1,대구광역시,수성구,상동,0.965926,1,겨울,700.0,NaN,NaN,NaN,52.500000,9025.0,5.0,338
1,ACCIDENT_39610,2022-01-01 01:00:00,토요일,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람,2022,1,1,1,대구광역시,수성구,지산동,0.965926,1,겨울,NaN,0.0,0.0,2.0,47.142857,52441.0,10.0,492


In [11]:
# 불필요 컬럼 제거
drop_columns = ['ID','도시','시간','시군구','사고일시','사고유형 - 세부분류','법규위반','가해운전자 차종','가해운전자 성별','가해운전자 연령',
       '가해운전자 상해정도','사망자수','중상자수','피해운전자 차종','피해운전자 성별',
      '피해운전자 연령','피해운전자 상해정도','경상자수','부상자수']
train_df.drop(columns = drop_columns,inplace=True)
test_df.drop(columns=['ID','도시','시간','사고일시','시군구'],inplace=True)

### 이상치 처리 및 결측값 처리

In [12]:
# 결측값 확인 
train_df.isnull().sum().sort_values(ascending=False)[:11]

어린이보호구역개수    18426
설치개수         13813
급지구분_3        6543
급지구분_2        6543
급지구분_1        6543
제한속도          2442
횡단보도개수          62
요일               0
기상상태             0
season           0
holiday          0
dtype: int64

In [13]:
test_df.isnull().sum().sort_values(ascending=False)[:11]

어린이보호구역개수    4961
설치개수         4029
급지구분_3       1928
급지구분_2       1928
급지구분_1       1928
제한속도          729
횡단보도개수         21
요일              0
holiday         0
season          0
cos_hour        0
dtype: int64

In [14]:
# 일단 interpolate
for df in [train_df,test_df]:
    df['어린이보호구역개수'] = df['어린이보호구역개수'].interpolate()
    df['급지구분_1'] = df['급지구분_1'].interpolate()
    df['급지구분_2'] = df['급지구분_2'].interpolate()
    df['급지구분_3'] = df['급지구분_3'].interpolate()
    df['설치개수'] = df['설치개수'].interpolate()
    df['제한속도'] = df['제한속도'].interpolate()
    df['횡단보도개수'] = df['횡단보도개수'].interpolate()
    
# 급지구분 interpolate하면 안 채워지는 게 1개 있어서 0으로 대체
test_df['급지구분_1'] = test_df['급지구분_1'].fillna(0)
test_df['급지구분_2'] = test_df['급지구분_2'].fillna(0)
test_df['급지구분_3'] = test_df['급지구분_3'].fillna(0)

In [15]:
# 결측값 확인 
train_df.isnull().sum().sort_values(ascending=False)[:5]," ---- " , test_df.isnull().sum().sort_values(ascending=False)[:5]

(요일           0
 기상상태         0
 어린이보호구역개수    0
 횡단보도개수       0
 제한속도         0
 dtype: int64,
 ' ---- ',
 요일           0
 holiday      0
 어린이보호구역개수    0
 횡단보도개수       0
 제한속도         0
 dtype: int64)

### 인코딩 및 스케일링

In [16]:
# 원핫 : 구

# train_df와 test_df의 기상상태 및 노면상태 열 선택
train_categorical_data = train_df[['구']]
test_categorical_data = test_df[['구']]

# OneHotEncoder 인스턴스 생성 및 fit_transform 수행
encoder = OneHotEncoder()
train_encoded = encoder.fit_transform(train_categorical_data)
test_encoded = encoder.transform(test_categorical_data)

# OneHotEncoder가 사용한 카테고리 목록을 가져와서 카테고리 이름을 열 이름으로 변환
feature_names = encoder.get_feature_names_out(['구'])

# 밀집 행렬로 변환 (선택 사항)
train_encoded_dense = train_encoded.toarray()
test_encoded_dense = test_encoded.toarray()

# 데이터프레임으로 변환 (선택 사항)
import pandas as pd

train_encoded_df = pd.DataFrame(train_encoded_dense, columns=feature_names, index=train_df.index)
test_encoded_df = pd.DataFrame(test_encoded_dense, columns=feature_names, index=test_df.index)

# 기존 열 제거
train_df = train_df.drop(['구'], axis=1)
test_df = test_df.drop(['구'], axis=1)

# 인코딩된 열 추가
train_df = pd.concat([train_df, train_encoded_df], axis=1)
test_df = pd.concat([test_df, test_encoded_df], axis=1)


In [17]:
# 타겟 인코딩 : 나머지
# Target encoding
from category_encoders.target_encoder import TargetEncoder

categorical_features = ['요일','도로형태','사고유형','동','기상상태', '노면상태','season']

for i in categorical_features:
    tr_encoder = TargetEncoder(cols=[i])
    train_df[i] = tr_encoder.fit_transform(train_df[i], train_df['ECLO'])
    test_df[i] = tr_encoder.transform(test_df[i])

### 기타 전처리

In [18]:
# 송규헌 : 연도값을 연도별 추세로 변경 (일단 임의로 넣어봄)
year_trend = dict(zip([2019,2020,2021,2022,2023],[4.8,4.6,4.4,4.2,4.0]))
train_df['연'] =  train_df['연'].map(lambda x:year_trend[x])
test_df['연'] =  test_df['연'].map(lambda x:year_trend[x])

### 데이터셋 분할

In [19]:
# 컬럼명 영어로 바꾸기
kr2eng = {'요일':'dayofweek', '기상상태':'weather', '도로형태':'road_type',
          '노면상태':'road_state', '사고유형':'accident_type', '연':'year',
          '동':'dong', '설치개수':'light_count', '제한속도':'speed_limit',
          '횡단보도개수':'cross_count', '어린이보호구역개수':'child_count',
          '사고발생횟수':'accident_count', '주차장지수':'parking_index',
          '구_남구':'gu_namgu', '구_달서구':'gu_dalseogu', 
          '구_달성군':'gu_dalsunggun', '구_동구':'gu_donggu', 
          '구_북구':'gu_bookgu', '구_서구':'gu_seogu',
       '구_수성구':'gu_suseonggu', '구_중구':'gu_junggu',
         '월':'month','일':'day','급지구분_1':'parking_1',
         '급지구분_2':'parking_2','급지구분_3':'parking_3'
         }

train_df.rename(columns=kr2eng,inplace=True)
test_df.rename(columns=kr2eng,inplace=True)

In [20]:
display(train_df), display(test_df)

,dayofweek,weather,road_type,road_state,accident_type,ECLO,year,month,day,dong,cos_hour,holiday,season,light_count,parking_1,parking_2,parking_3,speed_limit,cross_count,child_count,accident_count,gu_namgu,gu_dalseogu,gu_dalsunggun,gu_donggu,gu_bookgu,gu_seogu,gu_suseonggu,gu_junggu
0,4.627926,4.712888,4.653990,4.712878,3.817650,5,4.8,1,1,4.282449,1.000000,1,4.659111,391.0,11.0,0.0,0.0,40.000000,5929.0,2.000000,131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,4.627926,4.779150,4.653990,4.712878,3.817650,3,4.8,1,1,4.738938,1.000000,1,4.659111,941.0,0.0,1.0,3.0,52.500000,10000.0,3.500000,678,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.627926,4.712888,4.653990,4.712878,3.817650,3,4.8,1,1,4.842715,0.965926,1,4.659111,473.0,0.0,5.0,4.0,55.000000,14641.0,5.000000,604,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4.627926,4.712888,4.653990,4.712878,4.944597,5,4.8,1,1,4.208920,0.866025,1,4.659111,534.0,0.0,9.0,5.0,38.000000,29929.0,11.000000,426,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,4.627926,4.712888,4.653990,4.712878,4.944597,3,4.8,1,1,4.549091,0.500000,1,4.659111,2064.0,0.0,1.0,0.0,40.000000,58564.0,10.666667,825,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,4.600906,4.712888,5.006142,4.712878,4.944597,3,4.4,12,31,4.913420,0.258819,0,4.659111,941.5,3.5,0.0,2.5,40.000000,2809.0,1.000000,104,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
39605,4.600906,4.712888,4.653990,4.712878,4.944597,3,4.4,12,31,4.293963,0.258819,0,4.659111,844.0,0.0,0.0,5.0,44.444444,100489.0,5.500000,1143,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
39606,4.600906,4.712888,5.006142,4.712878,4.944597,10,4.4,12,31,4.648867,0.707107,0,4.659111,167.0,0.0,1.0,0.0,41.111111,79524.0,10.000000,618,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
39607,4.600906,4.712888,4.065962,4.712878,4.944597,3,4.4,12,31,4.672002,0.866025,0,4.659111,210.0,0.0,0.0,1.0,60.000000,7921.0,14.500000,125,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


,dayofweek,weather,road_type,road_state,accident_type,year,month,day,dong,cos_hour,holiday,season,light_count,parking_1,parking_2,parking_3,speed_limit,cross_count,child_count,accident_count,gu_namgu,gu_dalseogu,gu_dalsunggun,gu_donggu,gu_bookgu,gu_seogu,gu_suseonggu,gu_junggu
0,4.920811,4.712888,5.006142,4.712878,3.817650,4.2,1,1,4.881657,9.659258e-01,1,4.659111,700.00,0.0,0.0,0.0,52.500000,9025.0,5.00,338,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,4.920811,4.712888,4.653990,4.712878,3.817650,4.2,1,1,4.563008,9.659258e-01,1,4.659111,760.25,0.0,0.0,2.0,47.142857,52441.0,10.00,492,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,4.920811,4.712888,5.006142,4.712878,4.944597,4.2,1,1,4.945578,5.000000e-01,1,4.659111,820.50,0.0,1.0,1.5,40.000000,1764.0,1.00,147,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4.920811,4.712888,4.653990,4.712878,4.944597,4.2,1,1,4.438172,5.000000e-01,1,4.659111,880.75,0.0,2.0,1.0,42.727273,28561.0,7.00,372,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,4.920811,4.712888,5.006142,4.712878,4.944597,4.2,1,1,4.738938,6.123234e-17,1,4.659111,941.00,0.0,1.0,3.0,52.500000,10000.0,9.75,678,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,4.920811,4.712888,6.555670,4.712878,4.944597,4.2,12,31,4.492281,-1.836970e-16,1,4.659111,5377.00,20.0,1.0,0.0,47.037037,291600.0,26.00,1749,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10959,4.920811,4.712888,4.653990,4.712878,4.944597,4.2,12,31,4.928144,-1.836970e-16,1,4.659111,4323.75,20.0,1.0,0.0,52.500000,18225.0,5.00,167,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10960,4.920811,4.712888,4.653990,4.712878,4.944597,4.2,12,31,5.355036,5.000000e-01,1,4.659111,3270.50,20.0,1.0,0.0,48.392857,1024.0,6.00,107,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10961,4.920811,4.712888,4.863035,4.712878,4.944597,4.2,12,31,5.079498,5.000000e-01,1,4.659111,2217.25,20.0,1.0,0.0,44.285714,33124.0,7.00,239,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


(None, None)

In [21]:
train_x = train_df[test_df.columns] # test 셋에만 있는 컬럼만 추출
train_y = np.log1p(train_df['ECLO'])
test_x = test_df

In [22]:
x_train, x_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [23]:
x_train

,dayofweek,weather,road_type,road_state,accident_type,year,month,day,dong,cos_hour,holiday,season,light_count,parking_1,parking_2,parking_3,speed_limit,cross_count,child_count,accident_count,gu_namgu,gu_dalseogu,gu_dalsunggun,gu_donggu,gu_bookgu,gu_seogu,gu_suseonggu,gu_junggu
19887,4.600906,4.712888,5.006142,4.712878,4.944597,4.6,6,12,4.678933,-1.836970e-16,0,4.714711,13.0,0.0,12.0,1.666667,46.250000,36.0,1.0,59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
17901,5.210258,4.712888,5.006142,4.712878,4.944597,4.6,4,19,4.438172,-9.659258e-01,1,4.764856,1216.5,0.0,2.0,1.000000,42.727273,28561.0,7.0,372,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
38196,4.600906,4.712888,4.653990,4.712878,4.944597,4.4,11,19,4.492281,-8.660254e-01,0,4.760698,5377.0,20.0,1.0,0.000000,47.037037,291600.0,26.0,1749,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30365,4.560688,4.712888,4.653990,4.712878,4.944597,4.4,4,1,4.839335,-5.000000e-01,0,4.764856,670.0,0.0,0.0,2.500000,48.666667,98596.0,14.0,361,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4562,4.920811,4.712888,5.006142,4.712878,4.944597,4.8,5,4,5.018124,9.659258e-01,1,4.764856,567.5,0.0,18.0,0.000000,50.000000,25600.0,9.0,938,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,4.600906,4.712888,4.653990,4.712878,4.944597,4.8,6,14,4.549091,1.000000e+00,0,4.714711,2064.0,0.0,1.0,0.000000,40.000000,58564.0,5.0,825,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
11284,4.627408,4.712888,4.863035,4.712878,4.944597,4.8,10,16,4.540412,-2.588190e-01,0,4.760698,405.0,0.0,1.0,1.000000,41.000000,40401.0,6.5,631,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
38158,4.560688,4.712888,4.653990,4.712878,3.817650,4.4,11,18,4.326648,-9.659258e-01,0,4.760698,273.0,0.0,0.0,7.000000,40.000000,34225.0,10.0,349,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
860,5.210258,4.712888,5.006142,4.712878,4.944597,4.8,1,27,4.648867,-5.000000e-01,1,4.659111,167.0,0.0,1.0,0.000000,41.111111,79524.0,8.0,618,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


### AUTOML

In [24]:
import matplotlib
matplotlib.use('Agg')

In [30]:
from supervised.automl import AutoML
automl = AutoML(mode="Optuna",
                algorithms = ['Random Forest', 'LightGBM', 'Xgboost', 
                              'CatBoost','Decision Tree','Extra Trees'],
                n_jobs = -1,total_time_limit=60000, 
                eval_metric="rmse", 
                ml_task = "regression",
               explain_level=2,
                results_path='./AutoML_001_Optuna_new/',
               verbose=1,
               random_state=42,
               stack_models=True)

In [31]:
automl.fit(train_x, train_y)

AutoML directory: ./AutoML_001_Optuna_new/
Expected computing time:
Time for tuning with Optuna: len(algorithms) * optuna_time_budget = 21600 seconds
There is no time limit for ML model training after Optuna tuning (total_time_limit parameter is ignored).
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Random Forest', 'LightGBM', 'Xgboost', 'CatBoost', 'Decision Tree', 'Extra Trees']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble', 'stack', 'ensemble_stacked']
* Step simple_algorithms will try to check up to 1 model
1_DecisionTree rmse 0.446086 trained in 41.37 seconds


[I 2023-12-09 10:42:41,221] A new study created in memory with name: no-name-fa1f1b7f-7849-4e9b-ad6f-8599d0beff7e


* Step default_algorithms will try to check up to 5 models
Optuna optimizes LightGBM with time budget 3600 seconds eval_metric rmse (minimize)


[I 2023-12-09 10:42:42,314] Trial 0 finished with value: 0.9924254247490734 and parameters: {'learning_rate': 0.025, 'num_leaves': 321, 'lambda_l1': 2.5348407664333426e-07, 'lambda_l2': 3.3323645788192616e-08, 'feature_fraction': 0.9063233107042159, 'bagging_fraction': 0.7207805142313962, 'bagging_freq': 5, 'min_data_in_leaf': 3, 'extra_trees': True}. Best is trial 0 with value: 0.9924254247490734.
[I 2023-12-09 10:42:42,986] Trial 1 finished with value: 0.9929033388407481 and parameters: {'learning_rate': 0.1, 'num_leaves': 1076, 'lambda_l1': 7.71800699380605e-05, 'lambda_l2': 4.17890272377219e-06, 'feature_fraction': 0.7282970324241945, 'bagging_fraction': 0.3976457038513679, 'bagging_freq': 3, 'min_data_in_leaf': 37, 'extra_trees': False}. Best is trial 0 with value: 0.9924254247490734.
[I 2023-12-09 10:42:45,187] Trial 2 finished with value: 0.9923829777697577 and parameters: {'learning_rate': 0.05, 'num_leaves': 1245, 'lambda_l1': 3.425445902633376e-07, 'lambda_l2': 3.850031979199

[I 2023-12-09 10:43:11,675] Trial 22 finished with value: 0.9857709489624257 and parameters: {'learning_rate': 0.025, 'num_leaves': 1704, 'lambda_l1': 9.932991200123183, 'lambda_l2': 8.892188973051864, 'feature_fraction': 0.7591677155227415, 'bagging_fraction': 0.5120913539964441, 'bagging_freq': 6, 'min_data_in_leaf': 93, 'extra_trees': False}. Best is trial 22 with value: 0.9857709489624257.
[I 2023-12-09 10:43:12,815] Trial 23 finished with value: 0.9862144423637466 and parameters: {'learning_rate': 0.025, 'num_leaves': 1678, 'lambda_l1': 0.3999969555700058, 'lambda_l2': 1.3630182316321857, 'feature_fraction': 0.7702614832972421, 'bagging_fraction': 0.41498713394169917, 'bagging_freq': 6, 'min_data_in_leaf': 91, 'extra_trees': False}. Best is trial 22 with value: 0.9857709489624257.
[I 2023-12-09 10:43:13,933] Trial 24 finished with value: 0.9876525180620555 and parameters: {'learning_rate': 0.025, 'num_leaves': 1743, 'lambda_l1': 0.4635850595641248, 'lambda_l2': 0.9746329439157845,

...

[I 2023-12-09 16:00:41,575] Trial 442 finished with value: 0.9864984424066998 and parameters: {'max_depth': 9, 'min_samples_split': 38, 'min_samples_leaf': 65, 'max_features': 0.6936657740343001}. Best is trial 314 with value: 0.9856781957924545.
[I 2023-12-09 16:00:49,105] Trial 443 finished with value: 0.9860182141456415 and parameters: {'max_depth': 11, 'min_samples_split': 45, 'min_samples_leaf': 36, 'max_features': 0.7835256488222642}. Best is trial 314 with value: 0.9856781957924545.
[I 2023-12-09 16:00:56,010] Trial 444 finished with value: 0.9861861518585734 and parameters: {'max_depth': 12, 'min_samples_split': 41, 'min_samples_leaf': 31, 'max_features': 0.6463232548161628}. Best is trial 314 with value: 0.9856781957924545.
[I 2023-12-09 16:01:04,785] Trial 445 finished with value: 0.9862948154693284 and parameters: {'max_depth': 13, 'min_samples_split': 48, 'min_samples_leaf': 43, 'max_features': 0.9159642940761731}. Best is trial 314 with value: 0.9856781957924545.
[I 2023-1

[I 2023-12-09 16:04:29,979] Trial 476 finished with value: 0.9860821756508241 and parameters: {'max_depth': 11, 'min_samples_split': 47, 'min_samples_leaf': 40, 'max_features': 0.9760974436465474}. Best is trial 314 with value: 0.9856781957924545.
[I 2023-12-09 16:04:35,864] Trial 477 finished with value: 0.9859816675885805 and parameters: {'max_depth': 12, 'min_samples_split': 45, 'min_samples_leaf': 36, 'max_features': 0.5383149292943868}. Best is trial 314 with value: 0.9856781957924545.
[I 2023-12-09 16:04:42,480] Trial 478 finished with value: 0.9859183714453127 and parameters: {'max_depth': 10, 'min_samples_split': 39, 'min_samples_leaf': 42, 'max_features': 0.8103873792193695}. Best is trial 314 with value: 0.9856781957924545.
[I 2023-12-09 16:04:50,549] Trial 479 finished with value: 0.9858550382748875 and parameters: {'max_depth': 11, 'min_samples_split': 96, 'min_samples_leaf': 39, 'max_features': 0.9406679432640883}. Best is trial 314 with value: 0.9856781957924545.
[I 2023-

[I 2023-12-09 16:08:28,393] Trial 510 finished with value: 0.9864586104301801 and parameters: {'max_depth': 10, 'min_samples_split': 40, 'min_samples_leaf': 29, 'max_features': 0.4069846071788162}. Best is trial 314 with value: 0.9856781957924545.
[I 2023-12-09 16:08:34,194] Trial 511 finished with value: 0.9860668300479625 and parameters: {'max_depth': 11, 'min_samples_split': 95, 'min_samples_leaf': 43, 'max_features': 0.555565639883466}. Best is trial 314 with value: 0.9856781957924545.
[I 2023-12-09 16:08:40,195] Trial 512 finished with value: 0.986222703410688 and parameters: {'max_depth': 10, 'min_samples_split': 42, 'min_samples_leaf': 47, 'max_features': 0.6371313584884356}. Best is trial 314 with value: 0.9856781957924545.
[I 2023-12-09 16:08:49,248] Trial 513 finished with value: 0.9861984694066872 and parameters: {'max_depth': 12, 'min_samples_split': 37, 'min_samples_leaf': 39, 'max_features': 0.9095370182360663}. Best is trial 314 with value: 0.9856781957924545.
[I 2023-12

6_Optuna_ExtraTrees rmse 0.444111 trained in 213.43 seconds
* Step ensemble will try to check up to 1 model
Ensemble rmse 0.443337 trained in 0.27 seconds
* Step stack will try to check up to 5 models
3_Optuna_Xgboost_Stacked rmse 0.443782 trained in 52.18 seconds
4_Optuna_CatBoost_Stacked rmse 0.443699 trained in 18.32 seconds
5_Optuna_RandomForest_Stacked rmse 0.443827 trained in 137.19 seconds
2_Optuna_LightGBM_Stacked rmse 0.443998 trained in 47.51 seconds
6_Optuna_ExtraTrees_Stacked rmse 0.443729 trained in 180.21 seconds
* Step ensemble_stacked will try to check up to 1 model
Ensemble_Stacked rmse 0.443277 trained in 0.5 seconds
AutoML fit time: 20432.74 seconds
AutoML best model: Ensemble_Stacked


AutoML(algorithms=['Random Forest', 'LightGBM', 'Xgboost', 'CatBoost',
                   'Decision Tree', 'Extra Trees'],
       eval_metric='rmse', explain_level=2, ml_task='regression', mode='Optuna',
       random_state=42, results_path='./AutoML_001_Optuna_new/',
       stack_models=True, total_time_limit=60000)

In [32]:
pred = automl.predict(test_x)

## 제출

In [33]:
submission = pd.read_csv(path+'sample_submission.csv')

In [34]:
submission['ECLO'] = np.expm1(pred)

In [35]:
submission.loc[ submission['ECLO'] < 0.0, 'ECLO'] = 0.0

In [36]:
min(submission['ECLO'])

2.7564759036763076

In [37]:
submission

,ID,ECLO
0,ACCIDENT_39609,3.921505
1,ACCIDENT_39610,3.542254
2,ACCIDENT_39611,4.780756
3,ACCIDENT_39612,4.185396
4,ACCIDENT_39613,4.558610
...,...,...
10958,ACCIDENT_50567,4.686300
10959,ACCIDENT_50568,4.355842
10960,ACCIDENT_50569,4.572876
10961,ACCIDENT_50570,4.655569


In [38]:
submit_path = "../data/submits/automl_1207_001_optuna_mode_new.csv"
submission.to_csv(submit_path, index=False)

In [40]:
min(np.expm1(pred)), max(np.expm1(pred))

(2.7564759036763076, 6.465465415189622)

In [ ]:
# # 데이콘 api 제출

# from dacon_submit_api import dacon_submit_api

# result = dacon_submit_api.post_submission_file(
#   submit_path,  # 제출 파일의 저장경로
#   '99e503e9d7311db9f4167d982d5823d7ef9427cc628aa63b96cf79d2920ba916', # API 토큰
#   '236193', # 대구공모전 ID
#   'song999', # 맘대루
#   '12.08 - 005_perform_mode. automl' )  # 제출 메모